In [1]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [2]:
df = pd.read_csv('Processed_Data.csv',lineterminator='\n')
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275000,Positive,aussie would happy series happen midst olympic...
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.133333,Negative,worst thing olympics finish whole week availab...
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.000000,Neutral,olympics play india hockey captain ranirampal ...
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.000000,Positive,see best moment tokyo close ceremony videoelep...
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.500000,Positive,fabulous olympics lovethebbc tokyo olympics bb...
...,...,...,...,...,...,...
140242,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.500000,Positive,congratulation winner participant olympics win...
140243,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.500000,Negative,hope wrong see tweet government
140244,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.000000,Neutral,tokyo pass baton paris strangest ever olympic ...
140245,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.000000,Neutral,paris plan deliver inclusive youth centre gend...


In [3]:
df = df.dropna()
df = df[df['processed_tweets'].apply(lambda x: x !="")]

In [4]:
df.describe()

,Unnamed: 0,sentiment
count,139715.000000,139715.000000
mean,70096.224908,0.176834
std,40503.120258,0.317648
min,0.000000,-1.000000
25%,35003.500000,0.000000
50%,70027.000000,0.000000
75%,105194.500000,0.375000
max,140246.000000,1.000000


converting the classes into mainly two category of Positive and Negetive. 

In [5]:
def ratio(x):
  if x=='Positive':
    return 1
  else:
    return 0;

In [6]:
label = df['class'].apply(ratio)
label

0         1
1         0
2         0
3         1
4         1
         ..
140242    1
140243    0
140244    0
140245    0
140246    1
Name: class, Length: 139715, dtype: int64

In [7]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['processed_tweets'])

sequences = tokenizer.texts_to_sequences(df['processed_tweets'])
data = pad_sequences(sequences, maxlen=50)

In [8]:
print(data.shape)

(139715, 50)


Building CNN+LSTM first. 

In [22]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [24]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(label), validation_split=0.4, epochs = 3)

Epoch 1/3
2620/2620 [==============================] - 64s 14ms/step - loss: 0.3391 - accuracy: 0.8684 - val_loss: 0.3310 - val_accuracy: 0.8761
Epoch 2/3
2620/2620 [==============================] - 35s 13ms/step - loss: 0.2477 - accuracy: 0.9086 - val_loss: 0.3320 - val_accuracy: 0.8734
Epoch 3/3
2620/2620 [==============================] - 35s 13ms/step - loss: 0.1866 - accuracy: 0.9313 - val_loss: 0.3774 - val_accuracy: 0.8718
